In [2]:
import os, sys
project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

from deepface.basemodels import VGGFace
tf_face = VGGFace.loadModel()

from deepface.extendedmodels import Gender
tf_gender = Gender.loadModel()

# import torch
# from core.face.vggface import VGGFace as PTVGGFace
# torch_model = PTVGGFace()

# weights_file = os.path.join(project_root, "pretrained/vggface.pth")
# torch_model.load_state_dict(torch.load(weights_file))

In [6]:
print(len(tf_face.layers), len(tf_gender.layers))
print(tf_gender.layers)
for face_layer, gender_layer in zip(tf_face.layers, tf_gender.layers):
  if not face_layer.get_weights(): continue
  print(face_layer.get_weights()[0].shape, gender_layer.get_weights()[0].shape)

38 39
(3, 3, 3, 64) (3, 3, 3, 64)
(3, 3, 64, 64) (3, 3, 64, 64)
(3, 3, 64, 128) (3, 3, 64, 128)
(3, 3, 128, 128) (3, 3, 128, 128)
(3, 3, 128, 256) (3, 3, 128, 256)
(3, 3, 256, 256) (3, 3, 256, 256)
(3, 3, 256, 256) (3, 3, 256, 256)
(3, 3, 256, 512) (3, 3, 256, 512)
(3, 3, 512, 512) (3, 3, 512, 512)
(3, 3, 512, 512) (3, 3, 512, 512)
(3, 3, 512, 512) (3, 3, 512, 512)
(3, 3, 512, 512) (3, 3, 512, 512)
(3, 3, 512, 512) (3, 3, 512, 512)
(7, 7, 512, 4096) (7, 7, 512, 4096)
(1, 1, 4096, 4096) (1, 1, 4096, 4096)
(1, 1, 4096, 2622) (1, 1, 4096, 2)


In [19]:
import cv2
import numpy as np
test_img = '/home/jixian/tmp/face_0_cropped.jpg'
img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
tf_img = np.expand_dims(img, axis=0)

tf_res = tf_model.predict(tf_img, verbose=0)[0].tolist()
print(tf_res[:10])
tf_res = tf_model.predict(tf_img, verbose=0)[0].tolist()
print(tf_res[:10])
# convert img to torch tensor

img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
img = img.astype(np.float32)
img = img.transpose(2, 0, 1)
torch_img = torch.from_numpy(img)

torch_model.eval()
with torch.no_grad():  
  torch_res = torch_model(torch_img).reshape(-1).detach().numpy().tolist()
  print(torch_res[:10])

  torch_res2 = torch_model(torch_img).reshape(-1).detach().numpy().tolist()
  print(torch_res2[:10])

[2.270510196685791, -0.28317683935165405, 0.6137429475784302, 1.8658781051635742, 2.337536573410034, 3.0819878578186035, 2.980890989303589, 1.5023161172866821, 2.195514678955078, 0.2994230389595032]
[2.270510196685791, -0.28317683935165405, 0.6137429475784302, 1.8658781051635742, 2.337536573410034, 3.0819878578186035, 2.980890989303589, 1.5023161172866821, 2.195514678955078, 0.2994230389595032]
[2.2696855068206787, -0.28304508328437805, 0.613831639289856, 1.8660333156585693, 2.337951183319092, 3.081942558288574, 2.98045015335083, 1.502403974533081, 2.1956019401550293, 0.29918134212493896]
[2.2696855068206787, -0.28304508328437805, 0.613831639289856, 1.8660333156585693, 2.337951183319092, 3.081942558288574, 2.98045015335083, 1.502403974533081, 2.1956019401550293, 0.29918134212493896]


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity([tf_res], [torch_res]))
print(cosine_similarity([tf_res], [torch_res2]))
print(cosine_similarity([torch_res], [torch_res2]))


[[0.99999999]]
[[0.99999999]]
[[1.]]
